# JSON and NTV data editor

This notebook presents with an example the functionality available to access, represent, update and transform JSON (or NTV) data.


## Summary
*(active link on jupyter Notebook or Nbviewer)*
- [Example](#Example)
- [Select data](#Select-data)
    - [JSON selecting](#JSON-selecting)
    - [NTV selecting](#NTV-selecting)
    - [Tree navigation](#Tree-navigation)
    - [Iterator](#Iterator)
- [Representation](#Representation)
- [Transform data](#Transform-data)
    - [Change structure](#Change-structure)
    - [Add semantic type](#Add-semantic-type)
- [Update data](#Update-data)
- [Restore initial structure](#Restore-initial-structure)

## References
- [JSON-NTV specification](https://datatracker.ietf.org/doc/draft-thomy-json-ntv/)
- [JSON-NTV classes and methods](https://loco-philippe.github.io/NTV/json_ntv.html)

This Notebook can also be viewed at [nbviewer](http://nbviewer.org/github/loco-philippe/NTV/tree/main/uses)

-----

In [1]:
from copy import copy
from pprint import pprint
from datetime import date, timedelta, datetime
from json_ntv import Ntv
from shapely import box

## Example

In [2]:
# This example is partially structured : only some data is defined with keywords
measure = {
    "location": [{"city": "paris"}, [4.1, 40.5]],
    "campaign": {
        "planning": {
            "period": "january 2023",
            "from": "2023-01-01",
            "to": "2023-01-20",
        },
        "property": "air pollutants",
        "my personnal comment": {
            "result": "success",
            "difficulty": "first air pollutants campaign in Paris",
        },
    },
    "pollutants": [{"NO2": "mcg/m3"}, {"CO": "mg/m3"}],
    "measurement": {
        "sample": "2023-01-10",
        "values": [["08:10:00", 21, 5], ["10:20:00", 10, 10], ["17:16:00", 30, 15]],
    },
}
n_measure = Ntv.obj(measure)  # NTV object
t_measure = n_measure.tree.dic_nodes  # NTV object as tree

## Select data

### JSON selecting
- selector is index for the array and key for the objects

In [3]:
# access to the measurement time of the second value
print(measure["measurement"]["values"][1][0])

# how to access to the 'result' of the 'campaign' (if i don't know the key)  ?

10:20:00


### NTV selecting
- selector is index or keys
- selector is a json-pointer
- selector is an absolute keys

In [4]:
n_measure["#/3/1/1/0"]

"10:20:00"

In [5]:
# access to the measurement time of the second value
print(n_measure["measurement"]["values"][1][0])
print(n_measure[3]["values"][1][0])
print(n_measure[3][1][1][0])
print(n_measure["#/measurement/values/1/0"])
print(n_measure["#/3/1/1/0"])
print(t_measure["values"].val[1][0])

# how to access to the 'period' of the 'campaign'  ?
print(t_measure["result"].val)
print(n_measure["campaign"][2]["result"].val)

"10:20:00"
"10:20:00"
"10:20:00"
"10:20:00"
"10:20:00"
"10:20:00"
success
success


### Tree navigation

In [6]:
# navigate in the tree
print(t_measure["from"].parent.parent["my personnal comment"].childs(nam=True))
print(t_measure["from"].parent.parent["my personnal comment"].childs(obj=True))

['result', 'difficulty']
[{'result': 'success'}, {'difficulty': 'first air pollutants campaign in Paris'}]


### Iterator
NtvTree is an iterator of the Ntv tree.
Four lists can be iterated:
- nodes : all the nodes
- dic_nodes : nodes with a name (dict : key = name, value = node)
- leaf_nodes : NtvSingle nodes
- inner_nodes : NtvList nodes

In [7]:
# <xx>.tree is an alias of NtvTree(<xx>)
print(
    "tree characteristics (size, height, breadth): ",
    n_measure.tree.size,
    n_measure.tree.height,
    n_measure.tree.breadth,
)
print("\nnames:\n", list(n_measure.tree.dic_nodes.keys()))
print("\nvalues:\n", [ntv.val for ntv in n_measure.tree.leaf_nodes])
print("\nlength:\n", [len(ntv) for ntv in n_measure.tree.inner_nodes])
print(
    "\nmute nodes:\n",
    [str(ntv.pointer()) for ntv in n_measure.tree if not ntv.name and not ntv.type_str],
)

tree characteristics (size, height, breadth):  33 4 21

names:
 ['location', 'city', 'campaign', 'planning', 'period', 'from', 'to', 'property', 'my personnal comment', 'result', 'difficulty', 'pollutants', 'NO2', 'CO', 'measurement', 'sample', 'values']

values:
 ['paris', 4.1, 40.5, 'january 2023', '2023-01-01', '2023-01-20', 'air pollutants', 'success', 'first air pollutants campaign in Paris', 'mcg/m3', 'mg/m3', '2023-01-10', '08:10:00', 21, 5, '10:20:00', 10, 10, '17:16:00', 30, 15]

length:
 [4, 2, 2, 3, 3, 2, 2, 2, 3, 3, 3, 3]

mute nodes:
 ['', '/location/1', '/measurement/values/0', '/measurement/values/1', '/measurement/values/2']


## Representation
- JSON with specific data
- with a tree

In [8]:
print("With expanded data:")
pprint(n_measure["location"].expand(), width=120)
print("\nWith values only:")
pprint(n_measure.to_obj(simpleval=True), width=140)
print("\nWith data codes:\n", repr(n_measure))  # l/s : NtvList/NtvSingle, N : with Name
print("\nWith JSON-array only:")
pprint(n_measure.to_obj(json_array=True), width=140)
print("\nWith binary data:\n", n_measure.to_obj(encoded=True, format="cbor"))

With expanded data:
{'E': 'NtvList',
 'N': 'location',
 'T': '',
 'V': [{'E': 'NtvSingle', 'N': 'city', 'T': 'json', 'V': 'paris'},
       {'E': 'NtvList',
        'N': '',
        'T': '',
        'V': [{'E': 'NtvSingle', 'N': '', 'T': 'json', 'V': 4.1},
              {'E': 'NtvSingle', 'N': '', 'T': 'json', 'V': 40.5}]}]}

With values only:
[['paris', [4.1, 40.5]],
 [['january 2023', '2023-01-01', '2023-01-20'], 'air pollutants', ['success', 'first air pollutants campaign in Paris']],
 ['mcg/m3', 'mg/m3'],
 ['2023-01-10', [['08:10:00', 21, 5], ['10:20:00', 10, 10], ['17:16:00', 30, 15]]]]

With data codes:
 {"location": [{"city": "paris"}, [4.1, 40.5]], "campaign": {"planning": {"period": "january 2023", "from": "2023-01-01", "to": "2023-01-20"}, "property": "air pollutants", "my personnal comment": {"result": "success", "difficulty": "first air pollutants campaign in Paris"}}, "pollutants": {"NO2": "mcg/m3", "CO": "mg/m3"}, "measurement": {"sample": "2023-01-10", "values": [["___"],

In [9]:
n_measure.to_mermaid("measure flowchart", disp=True)

## Transform data
- change structure
- add semantic type

### Change structure

In [10]:
# transform a NtvList into a NtvSingle
n_measure["location"][1] = n_measure["location"][1].to_ntvsingle(
    name="coord", typ="point"
)
# move a branch of the tree
n_measure["measurement"].insert(0, n_measure["pollutants"])

### Add semantic type

In [11]:
# add 'type' attribute
t_measure["from"].set_type("date")
t_measure["to"].set_type("date")
t_measure["sample"].set_type("date")
for val in t_measure["values"]:
    val[0].set_type("time")

In [12]:
# type is included in the JSON name
pprint(n_measure.to_obj(), width=140)

{'campaign': {'my personnal comment': {'difficulty': 'first air pollutants campaign in Paris', 'result': 'success'},
              'planning': {'from:date': '2023-01-01', 'period': 'january 2023', 'to:date': '2023-01-20'},
              'property': 'air pollutants'},
 'location': {'city': 'paris', 'coord:point': [4.1, 40.5]},
 'measurement': {'pollutants': {'CO': 'mg/m3', 'NO2': 'mcg/m3'},
                 'sample:date': '2023-01-10',
                 'values': [[{':time': '08:10:00'}, 21, 5], [{':time': '10:20:00'}, 10, 10], [{':time': '17:16:00'}, 30, 15]]}}


In [13]:
n_measure.to_mermaid("add semantic types", disp=True)

## Update data
- JSON value can be converted in object matching to the type

In [14]:
# convert JSON value into object
o_measure = n_measure.to_obj_ntv()
pprint(o_measure.to_obj(fast=True), width=140)

{'campaign': {'my personnal comment': {'difficulty': 'first air pollutants campaign in Paris', 'result': 'success'},
              'planning': {'from': datetime.date(2023, 1, 1), 'period': 'january 2023', 'to': datetime.date(2023, 1, 20)},
              'property': 'air pollutants'},
 'location': {'city': 'paris', 'coord': <POINT (4.1 40.5)>},
 'measurement': {'pollutants': {'CO': 'mg/m3', 'NO2': 'mcg/m3'},
                 'sample': datetime.date(2023, 1, 10),
                 'values': [[datetime.time(8, 10), 21, 5], [datetime.time(10, 20), 10, 10], [datetime.time(17, 16), 30, 15]]}}


In [15]:
# update objects : add one day to the dates, add one hour to the times, change point into box
for ntv in o_measure.tree.leaf_nodes:
    if ntv.type_str == "date":
        ntv.set_value(ntv.val + timedelta(days=1))
    if ntv.type_str == "time":
        ntv.set_value(
            (datetime.combine(date.today(), ntv.val) + timedelta(hours=1)).time()
        )
    if ntv.type_str == "point":
        r = 0.01
        ntv.set_value(
            box(
                round(ntv.val.x - r, 2),
                round(ntv.val.y - r, 2),
                round(ntv.val.x + r, 2),
                round(ntv.val.y + r, 2),
            ),
            fast=True,
        )
        ntv.set_type("polygon")

In [16]:
# convert object into JSON value
n_measure2 = o_measure.to_json_ntv()

In [17]:
pprint(n_measure2.to_obj(), width=140)

{'campaign': {'my personnal comment': {'difficulty': 'first air pollutants campaign in Paris', 'result': 'success'},
              'planning': {'from:date': '2023-01-02', 'period': 'january 2023', 'to:date': '2023-01-21'},
              'property': 'air pollutants'},
 'location': {'city': 'paris', 'coord:polygon': [[[4.11, 40.49], [4.11, 40.51], [4.09, 40.51], [4.09, 40.49], [4.11, 40.49]]]},
 'measurement': {'pollutants': {'CO': 'mg/m3', 'NO2': 'mcg/m3'},
                 'sample:date': '2023-01-11',
                 'values': [[{':time': '09:10:00'}, 21, 5], [{':time': '11:20:00'}, 10, 10], [{':time': '18:16:00'}, 30, 15]]}}


In [18]:
n_measure2.to_mermaid("update data", disp=True)

## Restore initial structure

In [19]:
# restore Point coordinate
coord = copy(o_measure["location"]["coord"])
coord.set_value(coord.val.centroid)
coord.set_type("point")
coord = coord.to_json_ntv()
coord.set_value([round(coord.val[0], 2), round(coord.val[1], 2)])
print(coord)

{"coord:point": [4.1, 40.5]}


In [20]:
# replace box with point coordinates
n_measure2["location"]["coord"].replace(coord)
# remove all the types
n_measure3 = n_measure2.no_type()
# replace coord by a list
n_measure3["location"][1] = n_measure3.tree.dic_nodes["coord"].to_ntvlist()
# remove coordinates name
n_measure3["location"][1].set_name()
# move pollutants in the tree
n_measure3.insert(2, n_measure3.tree.dic_nodes["pollutants"])

n_measure3.to_mermaid("update data", disp=True)

In [21]:
pprint(n_measure3.to_obj(), width=140)

{'campaign': {'my personnal comment': {'difficulty': 'first air pollutants campaign in Paris', 'result': 'success'},
              'planning': {'from': '2023-01-02', 'period': 'january 2023', 'to': '2023-01-21'},
              'property': 'air pollutants'},
 'location': [{'city': 'paris'}, [4.1, 40.5]],
 'measurement': {'sample': '2023-01-11', 'values': [['09:10:00', 21, 5], ['11:20:00', 10, 10], ['18:16:00', 30, 15]]},
 'pollutants': {'CO': 'mg/m3', 'NO2': 'mcg/m3'}}
